# The Type of Player Contributes The Most Data To The PLAICraft Research

## Introduction To Our Project

In [5]:
library(tidyverse)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages



In [7]:
url <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"

players <- read_csv(url)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
options(repr.plot.width = 8, repr.plot.height = 7)

players_wrangled <- players |>
                        select(- hashedEmail, - individualId, - organizationName) |>
                        group_by(experience) |>
                        filter(played_hours >= 0.1)

## Methods and Results

In [18]:
set.seed(1)

players_experience <- players_wrangled |>
    select(experience, age, played_hours) # Dataframe for Experience Classification

experience_split <- initial_split(players_experience, prop = 0.75, strata = experience)
experience_train <- training(experience_split)
experience_test <- testing(experience_split)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

In [19]:
experience_vfold <- vfold_cv(experience_train, v = 5, strata = experience)

In [17]:
experience_recipe <- recipe(experience ~ age + played_hours, data = exp_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_fit <- workflow() |>
  add_recipe(experience_recipe) |>
  add_model(knn_spec) |>
  fit(data = exp_subtrain)

Warning message:
“tune samples were requested but there were 59 rows in the data. 54 will be used.”
